## IMPORT LIBRARIES

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

UsageError: Line magic function `%tensorflow_version` not found.


In [3]:
import os
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN
import IPython.display as ipd
import json
import pretty_midi
from google.cloud import storage
from google.oauth2 import service_account

## PATH TO SAMPLE, TRAINING AND TEST DATA

In [4]:
MUSIC_TEST_OUTPUT = 'output'
MUSIC_SAMPLE_PATH = 'sample'
JSON_DATA_FILE = 'processed_data.json'
META_DATA_FILE = 'meta_data.json'
BUCKET_NAME = "bucket-piano-data"
MUSIC_TRAIN_PATH = 'gs://{}/train'.format(BUCKET_NAME)
MUSIC_TEST_PATH = 'gs://{}/{}'.format(BUCKET_NAME, META_DATA_FILE)
MUSIC_VALIDATION_PATH = 'gs://{}/validation'.format(BUCKET_NAME)
META_DATA_PATH = 'gs://{}/train'.format(BUCKET_NAME)

## MODEL PARAMETERS

In [5]:
RNN_CELL_SIZE = 256
BATCH_SIZE = 32
LEARNING_RATE = 1e-3
NUM_EPOCHS = 5
DECAY_RATE = 1e-5
LOSS_FN = 'binary_crossentropy'
METRICS = ['accuracy']
DROPOUT_RATE = 0.2

## LOAD TRAINING DATA

In [6]:
credentials = service_account.Credentials.from_service_account_file('piano-key-generation.json')
storage_client = storage.Client(project='piano-key-generation', credentials=credentials)
bucket = storage_client.get_bucket('bucket-piano-data')

In [7]:
blob = bucket.get_blob('meta_data.json') 
meta_data = json.loads(blob.download_as_string())

In [8]:
blob = bucket.get_blob('maestro-v3.0.0.json') 
file_data = json.loads(blob.download_as_string())

## AUDIO PARAMETERS

In [9]:
NUM_UNIQUE_NOTES = meta_data['num_unique_notes']
KEY_FPS = meta_data['key_fps']
HOP_LENGTH = meta_data['hop_length']
WINDOW_SIZE_IN_KEYS = meta_data['windows_size_in_keys']
SAMPLE_RATE = KEY_FPS * HOP_LENGTH
FRAME_SIZE = WINDOW_SIZE_IN_KEYS * HOP_LENGTH
MAX_PIANO_FREQ = meta_data['max_piano_freq']
SEQ_DURATION_SECS = meta_data['seq_length_in_mins']*60
SEQ_LENGTH = meta_data['seq_length']//5
NUM_UNIQUE_KEYS = 128


In [10]:
notes_to_token = meta_data['notes_to_tokens_dict']
token_to_notes = meta_data['tokens_to_notes_dict']
token_to_notes['0']='0'

## GET FILENAMES

In [11]:
def get_file_names(file_data, bucket, instance_type='train', seq_length=SEQ_DURATION_SECS):
    file_names = []
    for blob in bucket.list_blobs(prefix=instance_type):
        blob = bucket.get_blob(blob.name)
        if blob is not None:
            file_names.append(blob.name)
    return file_names

In [12]:
train_file_names = get_file_names(file_data, bucket, 'train', SEQ_DURATION_SECS)
test_file_names = get_file_names(file_data, bucket, 'test', SEQ_DURATION_SECS)
validation_file_names = get_file_names(file_data, bucket, 'validation', SEQ_DURATION_SECS)

In [ ]:
print(len(train_file_names))
print(len(test_file_names))
print(len(validation_file_names))

## Define Custom Data Generator

In [37]:
class CustomDataGenerator(keras.utils.Sequence):
    def __init__(self, bucket, file_names, token_to_notes, meta_data=meta_data, 
                    batch_size=BATCH_SIZE, num_unique_keys=NUM_UNIQUE_KEYS, seq_length=SEQ_LENGTH):
        self.bucket = bucket
        self.batch_size = batch_size
        self.meta_data = meta_data
        self.num_samples = len(file_names)
        self.file_names = file_names
        self.num_unique_keys = num_unique_keys
        self.token_to_notes = token_to_notes
        self.seq_length = SEQ_LENGTH
        
    def __len__(self):
        return (np.ceil(self.num_samples / float(self.batch_size))).astype(np.int)
    
    def convert_label_to_binary(self, output_labels):
        output_binary = []
        for sample in range(output_labels.shape[0]):
            sample_output = output_labels[sample,:]
            sample_output_binary = np.zeros((sample_output.size, self.num_unique_keys))
            for time_step in range(sample_output.size):
                notes_at_time_step = [int(note) for note in self.token_to_notes[str(sample_output[time_step])].split(',')]
                for note in notes_at_time_step:
                    sample_output_binary[time_step, note]=1
            output_binary.append(sample_output_binary)
        output = np.array(output_binary)
        return np.array(output_binary)
    
    def __getitem__(self, idx):
        batch_x = []
        batch_y = []
        start_idx = self.batch_size * idx
        for file in self.file_names[start_idx:start_idx+self.batch_size]:
            blob = bucket.get_blob(file)
            json_data = json.loads(json.loads(blob.download_as_string()))
            batch_x.append(json_data['input'])
            batch_y.append(json_data['output'])
        return np.array(batch_x), self.convert_label_to_binary(np.array(batch_y))

In [38]:
training_batch_generator = CustomDataGenerator(bucket, train_file_names, token_to_notes)
validation_batch_generator = CustomDataGenerator(bucket, validation_file_names, token_to_notes)

## A SINGLE BASIC RNN CELL

In [39]:
ipd.Image("images/rnn_vanilla.png")

## SPECIFY MODEL ARCHITECTURE

In [40]:
model = Sequential()
model.add(SimpleRNN(RNN_CELL_SIZE, activation='relu', return_sequences=True, input_shape=(SEQ_LENGTH, MAX_PIANO_FREQ), stateful=True,
                   batch_input_shape=(BATCH_SIZE, SEQ_LENGTH, MAX_PIANO_FREQ)))
# model.add(Dropout(DROPOUT_RATE))
model.add(Dense(NUM_UNIQUE_KEYS))
model.build()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (32, 2400, 256)           590080    
                                                                 
 dense (Dense)               (32, 2400, 128)           32896     
                                                                 
Total params: 622,976
Trainable params: 622,976
Non-trainable params: 0
_________________________________________________________________


## CHOOSE OPTIMIZER AND COMPILE MODEL

In [41]:
optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE, decay=DECAY_RATE)
model.compile(loss=LOSS_FN, optimizer=optimizer, metrics=METRICS)

## TRAIN MODEL

In [ ]:
model.fit_generator(generator=training_batch_generator, epochs=NUM_EPOCHS, validation_data=validation_batch_generator)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


## EVALUATE MODEL AND MAKE PREDICTION

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy on test data: %.2f%%" % (scores[1]*100))

In [ ]:
y_predict = model.predict(X_test)
print(y_predict.shape)

## CONVERT PREDICTION TO MIDI

In [ ]:
def convert_prediction_to_midi(indx, output, token_to_notes=test_token_to_notes, note_vel_mean_var=test_note_velocity_mean_var):
    assert (output.shape[-1] == NUM_UNIQUE_NOTES)
    notes_seq = [token_to_notes[str(x)] for x in output.argmax(1)]
    instrument = pretty_midi.Instrument(0, name='piano')
    note_details = {}
    pm = pretty_midi.PrettyMIDI()
    
    for index, note in enumerate(notes_seq):
        if note != '':
            notes = note.split(',')
            for i in notes:
                note_vel = np.random.normal(note_vel_mean_var[indx][0][int(i)], note_vel_mean_var[indx][1][int(i)])
                if int(i) not in note_details:
                    note_details[int(i)]=[[index], [index], [note_vel]]
                else:
                    if note_details[int(i)][1][-1]==(index-1):
                        note_details[int(i)][1][-1]=index
                    else:
                        note_details[int(i)][0].append(index)
                        note_details[int(i)][1].append(index)
                        note_details[int(i)][2].append(note_vel)

    for note in note_details:
        for start_time, end_time, note_velocity in zip(note_details[note][0], note_details[note][1], note_details[note][2]):
            pretty_note = pretty_midi.Note(
                velocity=int(note_velocity),
                pitch=note,
                start=start_time/KEY_FPS,
                end=end_time/KEY_FPS
            )
            instrument.notes.append(pretty_note)
    
    instrument.notes.sort(key=lambda x: x.start)
    pm.instruments.append(instrument)
    return pm

In [ ]:
file_indx = 0
pmidi = convert_prediction_to_midi(file_indx, y_predict[file_indx])
pmidi.write(os.path.join(MUSIC_TEST_OUTPUT, test_data['filenames'][file_indx]+'.midi'))